In [1]:
%load_ext autoreload
%autoreload 2

import sys
import json
import re
from datetime import datetime
from gridfs import GridFS, NoFile
from pymongo import MongoClient
from bson.objectid import ObjectId

sys.path.append('../')

from db_interface import addMagresFile, databaseSearch, removeMagresFiles, editMagresFile
from db_schema import magresVersionSchema, magresIndexSchema

### Operating on the database

This databook is meant for admin operations on the database and testing of new changes. Important: do *not* use carelessly, because permanent data loss is a very real possibility.

There are two options for usage, and the default is to use a local copy of the database. This works by having MongoDB installed and simply running a `mongod` instance in another terminal. In that case, testing can be freely done on a simple toy database. If `use_local` is set to `False`, though, the connection is established with the remote database, and all changes will be permanent. Double check all your code before running it!

In [2]:
use_local = True

if use_local:
    db_url = 'localhost'
    db_port = 27017
else:
    db_url = ''
    db_port = 27017
    
client = MongoClient(host=db_url, port=db_port)
ccpnc = client.ccpnc
magresIndex = ccpnc.magresIndex
magresMetadata = ccpnc.magresMetadata
magresFilesFS = GridFS(ccpnc, 'magresFilesFS')

In [3]:
def listAllKeys():
    f = magresIndex.find({}).next()
    print(f.keys())

In [4]:
def listAllFiles(keys=None, validate=False):
    if keys is not None:
        print('\t'.join(keys))
    for f in magresIndex.find({}):
        line = ''
        if keys is None:
            line = str(f)
        else:
            line = '\t'.join([str(f[k]) for k in keys])
        if validate:
            val = 'Valid'
            try:
                fc = dict(f) 
                del(fc['_id']) # Not included in the schema
                magresIndexSchema.validate(fc)
            except Exception as e:
                val = 'Not valid ({0})'.format(e)
            line += '\t' + val
        print(line)

In [7]:
listAllFiles(['chemname'], True)

chemname
Ala	Not valid (Missing keys: 'chemform', 'license')
Ala2	Not valid (Missing keys: 'chemform', 'license')
Thing	Not valid (Missing keys: 'chemform', 'license')
Ethanol	Not valid (Missing keys: 'chemform', 'license')
Test	Valid
